In [94]:
import requests, json, pickle, time
import os
from telegram.ext import *
import telegram
import pandas as pd
import time
import json

In [83]:
'https://www.binance.com/gateway-api/v1/friendly/pos/union?pageSize=50&pageIndex=1&status=ALL'

'https://www.binance.com/bapi/earn/v1/friendly/lending/project/list?pageSize=100&pageIndex=1&status=ALL'
defi_URL = 'https://www.binance.com/bapi/earn/v1/friendly/defi-pos/union?pageSize=50&pageIndex=1&status=ALL'
locked_URL = 'https://www.binance.com/gateway-api/v1/friendly/pos/union?status=ALL&pageSize=100'


URL = os.getenv("URL")
bot_token = os.getenv("bot_token")
bot_chatID = os.getenv("bot_chatID")

In [3]:
API = "1851628250:AAHPMfkpTEJpzWdzsEaUb3kU-VjUhiToMqE"

In [4]:
bot = telegram.Bot(token=API)
bot.sendMessage(chat_id = 1730011637, text = 'Hey there!')

# locked
response = json.loads(requests.get(locked_URL).text)["data"]
result = {}
for item in response:
    duration = []
    for asset in item["projects"]:
        duration.append(asset["duration"])
    result[item["asset"]] = duration
json.dump(result, open("data/locked.json", "w"))


# defi
response = json.loads(requests.get(defi_URL).text)["data"]
result = []
for item in response:
    result.append(item["asset"])
json.dump(result, open("data/defi.json", "w"))

In [156]:
def add_user(update):
    chat_id = update.message.chat_id
    df = pd.read_csv("data/users.csv")
    if chat_id not in df.id.values:
        username = update.effective_user.username
        firstname = update.effective_user.first_name
        lastname = update.effective_user.last_name
        with open("data/users.csv", "a+") as f:
            f.write(f"\n{time.asctime()},{chat_id},{username},{firstname},{lastname}")
            f.close()
        update.message.reply_text("Welcome new user !")

def subscribe_error():
    update.message.reply_text(
            "Please enter at least one currency to subscribe ! \nIf you need some /help."
    )

def get_user(chat_id):
    df = pd.read_csv("data/locked.csv")
    locked = df[df.id == chat_id]
    
    df = pd.read_csv("data/defi.csv")
    defi = df[df.id == chat_id]
    return locked, defi

In [176]:
def responses(input_text):
    return input_text

def start_command(update, context):
    print(context)
    add_user(update)
    help_command(update, context)

def help_command(update, context):
    update.message.reply_text(open("text/help.txt", "r").read())

def settings_command(update, context):
    chat_id = update.message.chat_id
    
    response = open("text/settings1.txt", "r").read()
    locked, defi = get_user(chat_id)
    for token in list(df_.token.drop_duplicates()):
        response += token \
            + " - {" \
            + ",".join([str(x) for x in list(locked[locked.token == token].duration)]) \
            + "}\n"
    response += "\n"
    response += open("text/settings2.txt", "r").read()
    response += "\n".join([str(x) for x in list(defi.token)])
    response += "\n"
    update.message.reply_text(response)
    
def list_command(update, context):
    response = "Currencies for locked staking:\n\n"
    dico = json.load(open("data/locked.json", "r"))
    for token in sorted(list(dico.keys())):
        response += token + " - {" + ",".join(dico[token]) + "}\n"
    
    response += "\nCurrencies for DeFi staking:\n\n"
    dico = json.load(open("data/defi.json", "r"))
    response += "\n".join(sorted(dico))
    update.message.reply_text(response)
    
def subscribe_command(update, context):
    liste = context.args
    if len(liste) == 0:
        subscribe_error()
    else:
        dico = json.load(open("data/locked.json", "r"))
        chat_id = update.message.chat_id
        locked, defi = get_user(chat_id)
        for item in liste:
            item = item.replace(" ", "").upper()
            durations = []
            if "-" in item:
                i = item.index("-")
                token = item[:i]
                if token not in dico.keys():
                        update.message.reply_text(
                            f"Token {token} not found"
                        )
                else:
                    for duration in item[i+1:].split(","):
                        if duration in dico[token]:
                            durations.append()
                        else:
                            update.message.reply_text(
                                f"Token {token} not found"
                            )
            else:
                token = item
                if token not in dico.keys():
                        update.message.reply_text(
                            f"Token {token} not found"
                        )
                else:
                    durations = dico[token]
            with open("data/locked.csv", "a+") as f:
                for duration in durations:
                    if ((locked.token==token) & (locked.duration==duration)).sum() == 0:
                        f.write(f"\n{chat_id},{token},{duration}")
                    else:
                        update.message.reply_text(
                            f"Token {token} already subscribed"
                        )
                f.close()
                
def test(update, context):
    pass
    
def error(update, context):
    print("Error")

In [ ]:
updater = Updater(API, use_context=True)
dp = updater.dispatcher

dp.add_handler(CommandHandler("start", start_command))
dp.add_handler(CommandHandler("help", help_command))
dp.add_handler(CommandHandler("settings", settings_command))
dp.add_handler(CommandHandler("list", list_command))
dp.add_handler(CommandHandler("subscribe", subscribe_command))
dp.add_handler(CommandHandler("test", test))


dp.add_error_handler(error)

updater.start_polling()
updater.idle()

Error


In [39]:
1730011637 in df.id.values

True

In [135]:
t = None

In [144]:
t

In [160]:
locked, defi = get_user(1730011637)

In [170]:
((locked.token == "test") & (locked.duration==45)).sum()

1

In [150]:
"BNB-30"[4:]

'30'

In [155]:
"30, 60".replace(" ", "").split(",")

['30', '60']